# Training on Wikiann

In this notebook, we use [MAD-X 2.0](https://arxiv.org/pdf/2012.15562.pdf) with a stacked language and task adapter setup to zero-shot cross-lingual transfer for NER.
We use a NER adapter from [AdapterHub.ml](https://adapterhub.ml/explore) pre-trained on the **English** portion of the [WikiAnn](https://www.aclweb.org/anthology/P17-1178.pdf) dataset and transfer to **Guarani** with a pre-trained language adapter.
This notebook is similar to the 'run_ner.py' example script in 'examples/pytorch/token-classification/'.

First, let's install 'adapter-transformers' and other required packages

In [1]:
!pip install -U adapter-transformers
!pip install datasets
!pip install seqeval

  Cloning https://github.com/hSterz/adapter-transformers.git (to revision dev/batch_split) to /tmp/pip-req-build-70gxfnbd
  Running command git clone -q https://github.com/hSterz/adapter-transformers.git /tmp/pip-req-build-70gxfnbd
  Running command git checkout -b dev/batch_split --track origin/dev/batch_split
  Switched to a new branch 'dev/batch_split'
  Branch 'dev/batch_split' set up to track remote branch 'dev/batch_split' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for adapter-transformers: filename=adapter_transformers-2.0.0-cp37-none-any.whl size=2099599 sha256=3feb17dca0940df00288ead1da3264cf964d2029f5524f92859b76a0c06627c8
  Stored in directory: /tmp/pip-ephem-wheel-cache-qwpi72pg/wheels/00/83/9b/5a3af174c4f3ad9b708896e050432422002e934b835dc9dd72
Successfully built adapter-transformers
  Found existing installation: adapter-transformers 2.0.0
    Uninstalling adap

Next, we initialize the tokenizer and the model with the correct labels.

In [2]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, AutoConfig
from datasets import load_dataset
from torch.utils.data import Dataset
import torch
import torch.nn.functional as F
from tqdm.notebook import tqdm
from torch import nn
import copy
#The labels for the NER task and the dictionaries to map the to ids or 
#the other way around
labels = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]
id_2_label = {id_: label for id_, label in enumerate(labels)}
label_2_id = {label: id_ for id_, label in enumerate(labels)}

model_name = "bert-base-multilingual-cased"
config = AutoConfig.from_pretrained(model_name, num_labels=len(labels), label2id=label_2_id, id2label=id_2_label)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, config=config)

print(model.get_labels())



Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']


Now, we load the task and the language adapter. For both adapters, we drop the adapter in the last layer following MAD-X 2.0. We then set both adapters as active adapters.

In [3]:
from transformers import AdapterConfig
target_language = "gn" # choose any language that a bert-base-multilingual-cased language adapter is available for
source_language = "en" # We support  "en", "ja", "zh", and "ar"

adapter_config = AdapterConfig.load(
    None,
    leave_out=[11]
)

model.load_adapter(
    "wikiann/" + source_language + "@ukp",
    config=adapter_config,
    load_as="wikiann",
)
    
lang_adapter_name = model.load_adapter(
    target_language + "/wiki@ukp",
    load_as=target_language,
    leave_out=[11],
)
# Set the adapters to be used in every forward pass
model.set_active_adapters([lang_adapter_name, "wikiann"])

Next, we can download the dataset and initialize the trainings arguments.

In [4]:
from datasets import load_dataset
from transformers import TrainingArguments

datasets = load_dataset('wikiann', target_language)

training_args = TrainingArguments(
    per_device_eval_batch_size=64,
    do_predict=True,
    output_dir="ner_models/madx/",
)

Reusing dataset wikiann (/root/.cache/huggingface/datasets/wikiann/gn/1.1.0/c0a0280cc1c835e2bb7db29f43ef89c8ea30e145b78c1ba2746d709fae3da112)


This method is taken from the example script 'run_ner.py'. It prepares the input tokens such that they are tokenized by the correct tokenizer and the labels are adapted to the new tokenization.

In [5]:
# This method is adapted from the huggingface transformers run_ner.py example script
# Tokenize all texts and align the labels with them.
def tokenize_and_align_labels(examples):
    text_column_name = "tokens"
    label_column_name = "ner_tags"
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=False,
        truncation=True,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

We apply the previous method to the test dataset to prepare it for prediction. 

In [6]:
from transformers import DataCollatorForTokenClassification
test_dataset = datasets["test"]
test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)

data_collator = DataCollatorForTokenClassification(tokenizer,)

Loading cached processed dataset at /root/.cache/huggingface/datasets/wikiann/gn/1.1.0/c0a0280cc1c835e2bb7db29f43ef89c8ea30e145b78c1ba2746d709fae3da112/cache-2d9847f420f61d2c.arrow


We use HuggingFace's `Trainer` class to evaluate zero-shot transfer on the WikiAnn test dataset.

In [7]:
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction
from datasets import load_metric
import numpy as np


# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    label_list = id_2_label

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Finally we can predict the labels for the test set and evaluate he predictions.

In [8]:
trainer.evaluate()

{'eval_accuracy': 0.7981220657276995,
 'eval_f1': 0.5714285714285714,
 'eval_loss': 0.9542072415351868,
 'eval_mem_cpu_alloc_delta': 10735616,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 352350208,
 'eval_precision': 0.4642857142857143,
 'eval_recall': 0.7428571428571429,
 'eval_runtime': 2.0225,
 'eval_samples_per_second': 49.444,
 'init_mem_cpu_alloc_delta': 842862592,
 'init_mem_cpu_peaked_delta': 0,
 'init_mem_gpu_alloc_delta': 754506240,
 'init_mem_gpu_peaked_delta': 0}